In [1]:
%run precarga_datos_librerias.py

# Codificacion de valores faltantes.

Notaciones más comunmente usadas:

In [2]:
common_na_strings = (
    "missing",
    "NA",
    "N A",
    "N/A",
    "#N/A",
    "NA ",
    " NA",
    "N /A",
    "N / A",
    " N / A",
    "N / A ",
    "na",
    "n a",
    "n/a",
    "na ",
    " na",
    "n /a",
    "n / a",
    " a / a",
    "n / a ",
    "NULL",
    "null",
    "",
    "?",
    "*",
    ".",
)

# Common NA numbers
common_na_numbers = (
    -9, -99, -999, -9999, 9999, 66, 77, 88, -1
)

# ¿Cómo encontrar los valores comunmente asociados a valores faltantes?

In [12]:
missing_data_example_df = pd.DataFrame.from_dict(dict(
    x = [1, 2, "NA", -99, -98, -99],
    y = ["A", "N/A", "NA", "E", "F", "G"],
    z = [-100, -99, -98, -101, -1, -1]
))



## Revisar los tipos de datos

In [5]:
missing_data_example_df.dtypes

x    object
y    object
z     int64
dtype: object

El primer indicio es observar entradas que no se corresponden con el tipo de dato.

## Valores unicos

In [8]:
missing_data_example_df.x.unique()

array([1, 2, 'NA', -99, -98], dtype=object)

Puede ser que esto facilite la visualizacion de valores faltantes anotados de forma erratica.

In [10]:
# Selecciona todas las no numericas
missing_data_example_df.select_dtypes(object)

,x,y
0,1,A
1,2,N/A
2,NA,NA
3,-99,E
4,-98,F
5,-99,G


El hecho de encontrar valores numericos en columnas categoricas puede dar indicio a presencia de valores faltantes.

## Sustitución de valores faltantes

### Sustitucion desde la lectura de datos.

Implica que ya conoces la estructura de tu dataset, y de como son almacenadas las variables en las bases de datos.

In [15]:
pd.read_csv(
    f"{raw}/missing_data_encoding_example.csv",
    na_filter=True,
    # Aniadir las representaciones a valores NaN
    na_values=[-99, -1]
)

,Unnamed: 0,x,y,z
0,0,1.0,A,-100.0
1,1,2.0,NaN,NaN
2,2,NaN,NaN,-98.0
3,3,NaN,E,-101.0
4,4,-98.0,F,NaN
5,5,NaN,G,NaN


## Sustitucion global

In [17]:
(
    missing_data_example_df.
    replace(
    to_replace=[-99, "NA"]
)
)

/tmp/ipykernel_9644/3693780630.py:3: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  replace(


,x,y,z
0,1,A,-100
1,2,N/A,-100
2,2,N/A,-98
3,2,E,-101
4,-98,F,-1
5,-98,G,-1


## Sustitucion dirigida
Para aplicar reglas de sustitucion sólo a una columna dirigida.

In [19]:
# En este caso se reemplaza -99 por np.nan solo en la columna x
(
    missing_data_example_df.
    replace(
        to_replace={
            "x":{
                -99: np.nan
            }
        }
    )
)

,x,y,z
0,1,A,-100
1,2,N/A,-99
2,NA,NA,-98
3,NaN,E,-101
4,-98,F,-1
5,NaN,G,-1


# Conversión de valores faltantes implicitos en explicitos.

Valores faltantes implicitos suceden cuando, por ejemplo, faltan LOS REGISTROS ENTEROS respecto a la medición estándar de los datos para ciertas variables cualitativas.

In [21]:
implicit_to_explicit_df = pd.DataFrame.from_dict(
    data={
        'name': ["Linda", "Linda", "Linda", "Fea"],
        'time': ['morning', 'afternoon', 'night', 'morning'],
        'value':[350, 310, np.nan, 320]
    }
)
implicit_to_explicit_df

,name,time,value
0,Linda,morning,350.0
1,Linda,afternoon,310.0
2,Linda,night,NaN
3,Fea,morning,320.0


Lo anterior es una mezcla de valores faltantes implicitos y explicitos

## Pivotar tabla de datos.

In [25]:
(
    implicit_to_explicit_df
    .pivot(
        index='name',
        columns='time',
        values='value'
    )
)

time,afternoon,morning,night
name,,,
Fea,NaN,320.0,NaN
Linda,310.0,350.0,NaN


In [29]:
itoe_pivoted = pd.pivot(
    data = implicit_to_explicit_df,
    index = 'name',
    columns = 'time',
    values = 'value'
)

In [31]:
itoe_pivoted.reset_index()

time,name,afternoon,morning,night
0,Fea,NaN,320.0,NaN
1,Linda,310.0,350.0,NaN


In [33]:
pd.melt(
    itoe_pivoted.reset_index(),
    id_vars = ["name"],
    value_vars = ["afternoon", "morning", "night"]
)

,name,time,value
0,Fea,afternoon,NaN
1,Linda,afternoon,310.0
2,Fea,morning,320.0
3,Linda,morning,350.0
4,Fea,night,NaN
5,Linda,night,NaN


Esto es lo contrario a hacer un melt de data frames.

## Cuantificar la ocurrencia de n tuplas.

In [36]:
(
    implicit_to_explicit_df
    .value_counts(
        subset=['name']
    )
    .reset_index(name = 'n')
    .query('n < 3')
)

,name,n
1,Fea,1


El observar que hay valores menores al estandar de mediciones por variable cuantitativa implica la presencia de valores faltantes.